Which charging station and charger type should they use? This file is made to answer that. 

In [ ]:
import pandas as pd

# 1. Load datasets
df_rates = pd.read_csv("/Users/your-name/EvoCharge/data/ca_county_prices/ca_county_rates.csv")
df_stations = pd.read_csv("/Users/your-name/EvoCharge/data/ca_county_prices/ev_charging_stations_county_prices.csv")

print("Rates loaded:", df_rates.shape)
print("Stations loaded:", df_stations.shape)

# 2. Standardize county column names
df_rates.rename(columns=str.lower, inplace=True)
df_stations.rename(columns=str.lower, inplace=True)

# Make sure "county_name" exists in both
rename_map = {
    "county": "county_name",
    "countyname": "county_name",
    "county_name": "county_name"
}
df_stations.rename(columns=rename_map, inplace=True)
df_rates.rename(columns=rename_map, inplace=True)

# 3. Perform merge + diagnostics
df_merged = df_stations.merge(
    df_rates[["county_name", "rate_per_kwh", "rate_tier"]],
    on="county_name",
    how="left",
    indicator=True
)

# 4. Merge diagnostics
print("\nMerge results:")
print(df_merged["_merge"].value_counts())

# 5. Drop merge indicator
df_merged.drop(columns="_merge", inplace=True)

# 6. Save
output_path = "/Users/kyleshiroma/EvoCharge/data/ca_county_prices/ev_charging_stations_merged.csv"
df_merged.to_csv(output_path, index=False)

print("\nSaved merged dataset to:", output_path)
print("Final columns:", list(df_merged.columns))


Rates loaded: (58, 5)
Stations loaded: (16455, 20)

Merge results:
_merge
both          9528
left_only     6927
right_only       0
Name: count, dtype: int64

Saved merged dataset to: /Users/kyleshiroma/EvoCharge/data/ca_county_prices/ev_charging_stations_merged.csv
Final columns: ['station name', 'street address', 'city', 'state', 'zip', 'ev level1 evse num', 'ev level2 evse num', 'ev dc fast count', 'ev network', 'ev connector types', 'access code', 'facility type', 'total_chargers', 'has_level1', 'has_level2', 'has_dc_fast', 'zip_code', 'county_name', 'electricity_rate_per_kwh', 'rate_tier_x', 'rate_per_kwh', 'rate_tier_y']


In [20]:
df_merged.head(10)


,station name,street address,city,state,zip,ev level1 evse num,ev level2 evse num,ev dc fast count,ev network,ev connector types,...,total_chargers,has_level1,has_level2,has_dc_fast,zip_code,county_name,electricity_rate_per_kwh,rate_tier_x,rate_per_kwh,rate_tier_y
0,LADWP - Truesdale Center,11797 Truesdale St,Sun Valley,CA,91352,0.0,57.0,2.0,SHELL_RECHARGE,CHADEMO J1772 J1772COMBO,...,59.0,False,True,True,91352.0,Los Angeles,0.3,Lowest,0.3,Lowest
1,Los Angeles Convention Center,1201 S Figueroa St,Los Angeles,CA,90015,0.0,7.0,0.0,Non-Networked,J1772,...,7.0,False,True,False,90015.0,Los Angeles,0.3,Lowest,0.3,Lowest
2,LADWP - John Ferraro Building,111 N Hope St,Los Angeles,CA,90012,0.0,338.0,12.0,Non-Networked,CHADEMO J1772 J1772COMBO,...,350.0,False,True,True,90012.0,Los Angeles,0.3,Lowest,0.3,Lowest
3,LADWP - Haynes Power Plant,6801 E 2nd St,Long Beach,CA,90803,0.0,19.0,1.0,Non-Networked,CHADEMO J1772 J1772COMBO,...,20.0,False,True,True,90803.0,Los Angeles,0.3,Lowest,0.3,Lowest
4,LADWP - Harbor Generating Station,161 N Island Ave,Wilmington,CA,90744,0.0,10.0,0.0,Non-Networked,J1772,...,10.0,False,True,False,90744.0,Los Angeles,0.3,Lowest,0.3,Lowest
5,LADWP - Sylmar West,13201 Sepulveda Blvd,Sylmar,CA,91342,0.0,2.0,0.0,Non-Networked,J1772,...,2.0,False,True,False,91342.0,Los Angeles,0.3,Lowest,0.3,Lowest
6,LADWP - EV Service Center,1630 N Main St,Los Angeles,CA,90012,0.0,46.0,1.0,Non-Networked,CHADEMO J1772,...,47.0,False,True,True,90012.0,Los Angeles,0.3,Lowest,0.3,Lowest
7,LADWP - Fairfax Center,2311 S Fairfax Ave,Los Angeles,CA,90016,0.0,13.0,0.0,Non-Networked,J1772,...,13.0,False,True,False,90016.0,Los Angeles,0.3,Lowest,0.3,Lowest
8,California Air Resources Board,9530 Telstar Ave,El Monte,CA,91731,0.0,3.0,0.0,Non-Networked,J1772,...,3.0,False,True,False,91731.0,Los Angeles,0.3,Lowest,0.3,Lowest
9,LADWP - Palmetto Center,1212 Palmetto St,Los Angeles,CA,90013,0.0,27.0,0.0,Non-Networked,J1772,...,27.0,False,True,False,90013.0,Los Angeles,0.3,Lowest,0.3,Lowest


In [29]:
def recommend_station(df, connector_type, prefer_fast=True, top_n=5):
    """
    EvoCharge Recommendation Engine (fixed for lowercase column names)
    """

    # map to lowercase columns
    df = df.rename(columns=lambda x: x.lower().replace('.', '').replace('_', ' ').strip())

    #step 1: filter by connector compatibility
    compatible = df[df["ev connector types"].str.contains(connector_type, case=False, na=False)]
    
    if compatible.empty:
        return f"No compatible stations found for connector type '{connector_type}'."

    #step 2: ranking method
    if prefer_fast:
        compatible = compatible.sort_values(
            by=["has dc fast", "has level2", "total chargers"],
            ascending=[False, False, False]
        )
    else:
        compatible = compatible.sort_values(
            by=["has level2", "total chargers"],
            ascending=[False, False]
        )

    #step 3: return top N stations
    return compatible.head(top_n)[[
        "station name",
        "street address",
        "city",
        "zip",
        "ev connector types",
        "ev level2 evse num",
        "ev dc fast count",
        "total chargers",
        "rate per kwh"  # optional for cost
    ]]


In [27]:
# Recommend top stations for a car with J1772 connector, preferring DC Fast
top_stations = recommend_station(df_merged, "J1772", prefer_fast=True)
print(top_stations)


                                            station name  \
2                          LADWP - John Ferraro Building   
697    Los Angeles Police Department - Main Street Pa...   
92                       LADWP - Valley Center Admin Lot   
4707   California Institute of Technology - Californi...   
12381                    Keck Hospital Parking Structure   

               street address         city    zip  \
2               111 N Hope St  Los Angeles  90012   
697             260 S Main St  Los Angeles  90012   
92           14401 Saticoy St     Van Nuys  91405   
4707   1248 E California Blvd     Pasadena  91125   
12381     1510 N San Pablo St  Los Angeles  90033   

                   ev connector types  ev level2 evse num  ev dc fast count  \
2            CHADEMO J1772 J1772COMBO               338.0              12.0   
697          CHADEMO J1772 J1772COMBO               100.0               4.0   
92           CHADEMO J1772 J1772COMBO                95.0               2.0   
4707